# NBA Statistics Predictor

## Player List
First we will define the year that we will train the model from <br>
We will then make a table of players that retired after that year

In [ ]:
# Install dependencies
%pip install string
%pip install pandas
%pip install -U scikit-learn
import pickle
from sklearn.preprocessing import LabelEncoder
import string
import pandas as pd
import random
import time
# SET THIS TO THE YEAR TO TRAIN THE MODEL FROM
year_from = 2023

In [ ]:


alphabet = string.ascii_lowercase

# Initialize an empty list to store player DataFrames
all_players = []

# Iterate through each letter in the alphabet to fetch player data
for letter in alphabet:
    random_seconds = random.randint(10, 20)
    print(f"Beginning letter {letter} and sleeping for {random_seconds} seconds")
    time.sleep(random_seconds)
    
    # Format the URL with the current letter
    url = f"https://www.basketball-reference.com/players/{letter}/"
    players_table = pd.read_html(url)[0]
    
    # Filter players based on retirement year
    players_table['To'] = pd.to_numeric(players_table['To'], errors='coerce')  # Convert 'To' column to numeric
    active_players = players_table[players_table['To'] > year_from]
    
    # Append the filtered DataFrame to the list
    all_players.append(active_players)

# Concatenate all player DataFrames into a single DataFrame
all_players_df = pd.concat(all_players, ignore_index=True)

# Save the DataFrame to a CSV file with the correct file name
file_name = f'players_list_{year_from}.csv'
all_players_df.to_csv(file_name, index=False)

print(f"Data saved to {file_name}")

# Player trainning configuration

Whether you are beginning to process the data or picking up where you left off this is all the configuration

In [6]:


# import the list of players
player_list = pd.read_csv('players_list_2023.csv') #change to correct file

num_players = len(player_list)


players_table = pd.DataFrame({}) # change to correct file

## Individual Player Data

Now there is a csv file in the directory with a list of players that retired after the year selected <br>
<br>
We will now itterate over each player and create a data frame with their game logs for their carrer <br>
The goal is to create a csv file for each all players and the games that they played <br><br>

### This block will contain helper functions for processing each player's data


In [7]:
# returns url that contains data about what years the player played
def get_player_years_played_url(row):
    player_name = row['Player'].lower().split()
    
    first_letter_last = player_name[1][0]
    unique_id = player_name[1][:5] + player_name[0][:2]
    return f"https://www.basketball-reference.com/players/{first_letter_last}/{unique_id}01.html"

# returns an array of the years that the player played in the NBA
def get_player_years_played(player_url):
    time.sleep(random.randint(3, 8))
    tables = pd.read_html(player_url)
    player_seasons = tables[1] #7 for player

    player_years_played = []
    # for each row of seasons
    for index, row in player_seasons.iterrows():
        season = row['Season']
        if season != "Career":
            season_year = int(season[:2] + season[-2:])
            if int(season_year) >= year_from:
                player_years_played.append(season_year)
        else:
            break
    return player_years_played

## Main function for pre-processing all the player stats
It is run for every table that the player has, it adds necessary rows and fills them with values it also cleans up values that are not numbers
<br>
It also hadles giving text data a numerical value


In [22]:
def process_player_career(player_url, years_played, player_name):
    player_data = pd.DataFrame()
    for year in years_played:
        print(f"processing row {year}")
        year_url = f"{player_url[:-5]}/gamelog/{year}"
        time.sleep(random.randint(3, 5))
        tables = pd.read_html(year_url)
        player_year_data = tables[7]
        # add additional rows
        player_year_data['Name'] = player_name * len(player_year_data)
        player_year_data['Injured'] = 0 * len(player_year_data)
        # remove unnecessary rows
        del player_year_data['Rk']
        del player_year_data['G']
        # add name to row
        player_year_data['Home/Away'] = player_year_data.iloc[:, 3]
        # add zeros where necessary
        replace_dict = {"Did Not Dress": 0, "Did Not Play": 0, "Inactive": 0}
        player_year_data = player_year_data.fillna(0)
        player_year_data = player_year_data.replace(replace_dict)
        for index, row in player_year_data.iterrows():
            row['Date'] = int(row['Date'].replace("-",""))
            row['Age'] = int(row['Age'].replace("-",""))
            row['MP'] = int(row['MP'].replace(":",""))
            if(row['MP'] == 0):
                row['Injured'] = 1
            if(row['Home/Away'] == "@"):
                row['Home/Away'] = 1
            else:
                row['Home/Away'] = 0
            row['Name'] = player_name
        players_table = players_table.append(df2, ignore_index=True)
        print(players_table)

            

            



In [23]:


print(f"staring to proccess player data for {num_players} players")

# itterates over every player
for index in range(3): # change this for production
    print(f"Player {index} of {num_players} players")
    # this is the player
    row = player_list.iloc[index]
    player_url = get_player_years_played_url(row)
    years_played = get_player_years_played(player_url)
    print(row['Player'])
    player_table = process_player_career(player_url, years_played, row['Player'])


    print(row['Player'])
    print(years_played)

"""
player_df = player_df.drop(['Rk', 'G'], axis=1)
player_df.rename(columns={'Unnamed: 7': 'PtDiff', 'Unnamed: 5': 'Home/Away',}, inplace=True)


for index, row in df.iterrows():
    print(f"Index: {index}")
    print(f"Name: {row['Name']}, Age: {row['Age']}, City: {row['City']}")
    print()  # Add a blank line for readability

print(player_df.columns)
"""


staring to proccess player data for 568 players
Player 0 of 568 players
Precious Achiuwa
processing row 2023


AttributeError: 'int' object has no attribute 'replace'